# Amazon SageMaker AI MLOps: from idea to production in six steps

이 6개의 노트북 시퀀스는 간단한 노트북에서 ML 아이디어를 개발하는 것부터 자동화된 모델 구축과 CI/CD 배포 파이프라인, 모델 모니터링을 갖춘 프로덕션 솔루션까지 안내합니다.

다음 단계를 하나씩 따라하세요: **아이디어에서 프로덕션까지 6단계:**
||||
|---|---|---|
|1. |노트북에서 실험 ||
|2. |SageMaker AI 처리 작업과 SageMaker SDK로 확장 ||
|3. |ML 파이프라인, 모델 레지스트리, 피처 스토어로 운영화 ||
|4. |모델 구축 CI/CD 파이프라인 추가 ||
|5. |모델 배포 파이프라인 추가 ||
|6. |모델 및 데이터 모니터링 추가 ||

[A/B 테스팅](https://docs.aws.amazon.com/sagemaker/latest/dg/model-validation.html), 사용자 정의 [처리](https://docs.aws.amazon.com/sagemaker/latest/dg/build-your-own-processing-container.html), [훈련](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo.html) 및 [추론](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-main.html) 컨테이너, [디버깅 및 프로파일링](https://docs.aws.amazon.com/sagemaker/latest/dg/train-debugger.html), [보안](https://docs.aws.amazon.com/sagemaker/latest/dg/security.html), [멀티 모델](https://docs.aws.amazon.com/sagemaker/latest/dg/multi-model-endpoints.html) 및 [멀티 컨테이너](https://docs.aws.amazon.com/sagemaker/latest/dg/multi-container-endpoints.html) 엔드포인트, 그리고 [직렬 추론 파이프라인](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-pipelines.html)과 같은 다른 SageMaker 기능과 ML 주제에 대한 추가 실습 예제도 있습니다. `additional-topics` 폴더의 노트북을 탐색하여 이러한 기능들을 테스트해보세요.

이 노트북과 워크샵의 모든 노트북을 실행하려면 JupyterLab에서 `Python 3` 커널을 사용하세요

## Star GitHub repository

In [ ]:
%%html

<a class="github-button" href="https://github.com/aws-samples/amazon-sagemaker-from-idea-to-production" data-color-scheme="no-preference: light; light: light; dark: dark;" data-icon="octicon-star" data-size="large" data-show-count="true" aria-label="Star Amazon SageMaker secure MLOps on GitHub">Star</a>
<script async defer src="https://buttons.github.io/buttons.js"></script>

### Click this button ^^^ above ^^^

## 설정
SageMaker Python SDK의 최신 버전을 가져오세요.

<div class="alert alert-info"> 💡 워크샵과 모든 노트북은 SageMaker Distribution Image 버전 <strong>3.6.1</strong>에서 테스트되었습니다. 호환성 문제가 발생하면 JupyterLab 스페이스를 중지하고 특정 버전의 SageMaker Distribution Image로 다시 시작할 수 있습니다.
</div>

![](img/jupyterlab-app-image.png)

In [ ]:
%pip install --upgrade pip "sagemaker<3" boto3

In [ ]:
%pip install --upgrade "mlflow>=2,<3" sagemaker-mlflow

In [ ]:
%pip install -q "sagemaker[local]<3"

### 패키지 가져오기

In [ ]:
import time
import os
import json
import boto3
import numpy as np  
import pandas as pd 
import sagemaker
from time import gmtime, strftime, sleep
import mlflow

(sagemaker.__version__,boto3.__version__, mlflow.__version__)

### Constants 설정

In [ ]:
# Get some variables you need to interact with SageMaker service
boto_session = boto3.Session()
region = boto_session.region_name
bucket_name = sagemaker.Session().default_bucket()
bucket_prefix = "from-idea-to-prod/xgboost"  
sm_session = sagemaker.Session()
sm_client = boto_session.client("sagemaker")
sm_role = sagemaker.get_execution_role()
dataset_file_local_path = "data/bank-additional/bank-additional-full.csv"

initialized = True

print(sm_role)

In [ ]:
# Store some variables to keep the value between the notebooks
%store bucket_name
%store bucket_prefix
%store sm_role
%store region
%store initialized
%store dataset_file_local_path

### 도메인 ID 가져오기
많은 SageMaker Python SDK와 boto3 SageMaker API 호출에서 이 `domain_id` 값이 필요합니다. 노트북 메타데이터 파일에는 `domain_id` 값이 포함되어 있습니다. 다음 코드는 노트북 메타데이터 파일에 액세스하고 `domain_id`를 가져오는 방법을 보여줍니다.

In [ ]:
NOTEBOOK_METADATA_FILE = "/opt/ml/metadata/resource-metadata.json"
domain_id = None

if os.path.exists(NOTEBOOK_METADATA_FILE):
    with open(NOTEBOOK_METADATA_FILE, "rb") as f:
        metadata = json.loads(f.read())
        domain_id = metadata.get('DomainId')
        space_name = metadata.get('SpaceName')
        print(f"SageMaker domain id: {domain_id}")

if not space_name:
    raise Exception(f"Cannot find the current space name. Make sure you run this notebook in a JupyterLab in the SageMaker Studio")
else:
    print(f"Space name: {space_name}")
    
r = sm_client.describe_space(DomainId=domain_id, SpaceName=space_name)
user_profile_name = r['OwnershipSettings']['OwnerUserProfileName']

assert(user_profile_name)
print(f"User profile: {user_profile_name}")

%store domain_id
%store space_name
%store user_profile_name

### 스페이스 세부 정보 가져오기
SageMaker API `DescribeSpace` 호출을 사용하여 스페이스 리소스 사양을 가져옵니다.

In [ ]:
r = boto3.client('sagemaker').describe_space(DomainId=domain_id, SpaceName=space_name)
resource_spec = r['SpaceSettings']['JupyterLabAppSettings']['DefaultResourceSpec']
sm_image = resource_spec.get('SageMakerImageArn', 'not defined')
sm_image_version = resource_spec.get('SageMakerImageVersionAlias', 'not defined')
print(f"""
SageMaker image: \033[1m{sm_image}\033[0m
SageMaker image version: \033[1m{sm_image_version}\033[0m
""")

In [ ]:
r

In [ ]:
%store sm_image
%store sm_image_version

### MLflow 추적 서버에 연결
AWS 주도 워크샵을 진행 중이거나 제공된 CloudFormation 템플릿을 사용하여 워크샵 환경을 프로비저닝한 경우, MLflow 서버가 실행 중이어야 합니다. MLflow 서버가 없는 경우 [개발자 가이드](https://docs.aws.amazon.com/sagemaker/latest/dg/mlflow-create-tracking-server.html)를 따르거나 다음 코드 셀을 실행하여 새로 생성하세요.

MLflow 추적 서버를 생성 및 관리하고 관리형 MLflow 실험을 작업하려면 SageMaker 실행 역할에 다음 권한이 연결되어 있어야 합니다:

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "sagemaker-mlflow:*",
                "sagemaker:CreateMlflowTrackingServer",
                "sagemaker:UpdateMlflowTrackingServer",
                "sagemaker:DeleteMlflowTrackingServer",
                "sagemaker:StartMlflowTrackingServer",
                "sagemaker:StopMlflowTrackingServer",
                "sagemaker:CreatePresignedMlflowTrackingServerUrl"
            ],
            "Resource": "*"
        }
    ]
}
```

실행 중인 MLflow 서버가 있는지 확인하려면 다음 코드를 실행하세요.

In [ ]:
# 계정에서 활성 MLflow 서버 찾기
tracking_servers = [s['TrackingServerArn'] for s 
                    in boto3.client("sagemaker").list_mlflow_tracking_servers()['TrackingServerSummaries']
                    if s['IsActive'] == 'Active']

if len(tracking_servers) < 1:
    print("You don't have any active MLflow servers. Trying to find a server in the status 'Creating'...")

    r = boto3.client("sagemaker").list_mlflow_tracking_servers(
        TrackingServerStatus='Creating',
    )['TrackingServerSummaries']

    if len(r) < 1:
        print("You don't have any MLflow server in the status 'Creating'. Run the next code cell to create a new one.")
        mlflow_arn = None
        mlflow_name = None
    else:
        mlflow_arn = r[0]['TrackingServerArn']
        mlflow_name = r[0]['TrackingServerName']
        print(f"You have an MLflow server {mlflow_arn} in the status 'Creating', going to use this one")
else:
    mlflow_arn = tracking_servers[0]
    mlflow_name = tracking_servers[0].split('/')[1]
    print(f"You have {len(tracking_servers)} running MLflow server(s). Get the first server ARN:{mlflow_arn}")

In [ ]:
# 새로운 MLflow 서버 생성
if not mlflow_arn:
    ts = strftime('%d-%H-%M-%S', gmtime())
    mlflow_name = f"mlflow-{domain_id}-{ts}"
    r = boto3.client("sagemaker").create_mlflow_tracking_server(
        TrackingServerName=mlflow_name,
        ArtifactStoreUri=f"s3://{bucket_name}/mlflow/{ts}",
        RoleArn=sm_role,
        AutomaticModelRegistration=True,
    )

    mlflow_arn = r['TrackingServerArn']
    print(f"Server creation request succeded. The server {mlflow_arn} is being created.")

<div style="border: 4px solid coral; text-align: center; margin: auto;">
MLflow 서버 생성에는 최대 25분이 소요될 수 있습니다. 기다릴 필요 없이 워크샵을 계속 진행하세요.
</div>

In [ ]:
(mlflow_arn, mlflow_name, mlflow.__version__)

In [ ]:
%store mlflow_arn
%store mlflow_name

## Studio 로컬 모드 활성화를 위한 Docker 설치
Amazon SageMaker Studio 애플리케이션은 로컬 모드를 사용하여 추정기, 프로세서, 파이프라인을 생성한 다음 로컬 환경에 배포하는 것을 지원합니다. 로컬 모드를 사용하면 Amazon SageMaker 관리형 훈련 또는 호스팅 환경에서 실
행하기 전에 머신러닝 스크립트를 테스트할 수 있습니다. Studio에서 현재 지원하는 Docker 작업을 이해하려면 [Amazon SageMaker Studio의 로컬 모드 지원](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-updated-local.html)을 참조하세요.

Studio 애플리케이션에서 로컬 모드를 사용하려면 JupyterLab 공간에 Docker를 설치해야 합니다.

### Docker 액세스가 활성화되어 있는지 확인

In [ ]:
# SageMaker 도메인에서 Docker가 활성화되어 있는지 확인
docker_settings = sm_client.describe_domain(DomainId=domain_id).get('DomainSettings', {}).get('DockerSettings')
docker_enabled = False

if docker_settings:
    if docker_settings.get('EnableDockerAccess') in ['ENABLED']:
        print(f"The docker access is ENABLED in the domain {domain_id}")
        docker_enabled = True

if not docker_enabled:
    raise Exception(f"You must enable docker access in the domain to use Studio local mode")

<div style="border: 4px solid coral; text-align: center; margin: auto;">
이전 코드 셀에서 Docker 액세스가 활성화되지 않았다는 예외가 발생한 경우, 액세스를 활성화해야 합니다. 다음 지침을 참조하여 활성화하는 방법을 확인하세요.
</div>

In [ ]:
print(f"Domain id: {domain_id}")

### SageMaker 도메인에 대한 Docker 액세스 활성화

<div class="alert alert-info">도메인에서 Docker 액세스가 활성화되지 않은 경우에만 이 섹션을 실행하면 됩니다.
</div>

실행 역할에 `sagemaker:UpdateDomain` 권한이 필요합니다. 노트북 실행 역할에 이 권한이 없기 때문에 이 노트북에서 도메인을 업데이트할 수 없습니다. 도메인 설정을 업데이트하려면 다음 옵션 중 하나를 사용할 수 있습니다.

#### 옵션 1: 노트북에서 `update_domain` 실행
노트북 실행 역할에 해당 권한이 있는 경우 노트북에서 다음 코드를 실행할 수 있습니다:

```python
import boto3

r = boto3.client('sagemaker').update_domain(
    DomainId=domain_id,
    DomainSettingsForUpdate={
        'DockerSettings': {
            'EnableDockerAccess':'ENABLED',
        }
    }
)
```

#### 옵션 2: 터미널에서 `aws sagemaker` CLI 실행
해당 권한 `sagemaker:UpdateDomain`이 있는 터미널에서 `AWS CLI`를 실행해야 합니다. 다음 명령을 실행하세요:

```
aws sagemaker update-domain --domain-id <DOMAIN-ID> --domain-settings-for-update DockerSettings={EnableDockerAccess='ENABLED'}
```

예를 들어, AWS 계정의 [AWS CloudShell](https://aws.amazon.com/blogs/aws/aws-cloudshell-command-line-access-to-aws-resources/)에서 위 명령을 실행할 수 있습니다.

In [ ]:
# 업데이트된 설정 확인
sm_client.describe_domain(DomainId=domain_id)['DomainSettings']

### Docker 설치

In [ ]:
%%bash

# see https://docs.docker.com/engine/install/ubuntu/#install-using-the-repository
sudo apt-get update
sudo apt-get install -y ca-certificates curl
sudo install -m 0755 -d /etc/apt/keyrings
sudo curl -fsSL https://download.docker.com/linux/ubuntu/gpg -o /etc/apt/keyrings/docker.asc
sudo chmod a+r /etc/apt/keyrings/docker.asc

# Add the repository to Apt sources:
echo \
  "deb [arch=$(dpkg --print-architecture) signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu \
  $(. /etc/os-release && echo "$VERSION_CODENAME") stable" | \
  sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
sudo apt-get update

## Currently only Docker version 20.10.X is supported in Studio: see https://docs.aws.amazon.com/sagemaker/latest/dg/studio-updated-local.html
# pick the latest patch from:
# apt-cache madison docker-ce | awk '{ print $3 }' | grep -i 20.10
VERSION_STRING=5:20.10.24~3-0~ubuntu-jammy
sudo apt-get install docker-ce-cli=$VERSION_STRING docker-compose-plugin -y

# validate the Docker Client is able to access Docker Server at [unix:///docker/proxy.sock]
docker version
# validate the Docker Compose plugin installed
docker compose version

# Install Compose Switch. this is a replacement to the Compose V1 docker-compose (python) executable.
# It translates the command line into Compose V2 docker compose then run the latter.
# this is needed by the sagemaker python sdk to run in local mode
# see https://github.com/docker/compose-switch
curl -fL https://raw.githubusercontent.com/docker/compose-switch/master/install_on_linux.sh | sudo sh
docker-compose version

## 데이터

이 예제는 UCI ML 저장소의 [마케팅 데이터셋](https://archive.ics.uci.edu/ml/datasets/bank+marketing)을 사용합니다:
> [Moro et al., 2014] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014

이 데이터는 포르투갈 은행 기관의 직접 마케팅 캠페인과 관련이 있습니다. 마케팅 캠페인은 전화 통화를 기반으로 했습니다. 종종 제품(은행 정기예금)이 가입('yes')될지 아닐('no')를 확인하기 위해 동일한 고객에게 한 번 이상의 연락이 필요했습니다.

데이터셋을 다운로드하고 압축을 해제하세요:

In [ ]:
!wget -P data/ -N https://archive.ics.uci.edu/static/public/222/bank+marketing.zip --no-check-certificate

In [ ]:
import zipfile

with zipfile.ZipFile("data/bank+marketing.zip", "r") as z:
    print("Unzipping bank+marketing...")
    z.extractall("data")

with zipfile.ZipFile("data/bank-additional.zip", "r") as z:
    print("Unzipping bank-additional...")
    z.extractall("data")

print("Done")

### 데이터 확인

In [ ]:
df_data = pd.read_csv(dataset_file_local_path, sep=";")

pd.set_option("display.max_columns", 500)  # View all of the columns
df_data  # show first 5 and last 5 rows of the dataframe

### S3에 데이터 업로드

In [ ]:
input_s3_url = sagemaker.Session().upload_data(
    path=dataset_file_local_path,
    bucket=bucket_name,
    key_prefix=f"{bucket_prefix}/input"
)
print(f"Upload the dataset to {input_s3_url}")

%store input_s3_url

## kernel 재시작

In [ ]:
# Restart kernel to get the packages
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

## 추가 워크샵 진행
다음 노트북들을 순서대로(예: 1-2-3-4...) 진행하여 워크샵을 계속할 수 있습니다.

특정 주제에 관심이 있다면 다음 플로우 다이어그램에 표시된 대로 일부 노트북을 독립적으로 실행할 수 있습니다:

![](img/workshop-flow.png)

1단계 [아이디어 개발](01-idea-development.ipynb) 또는 3단계 [SageMaker 파이프라인](03-sagemaker-pipeline.ipynb)부터 시작하세요.

## 추가 리소스

### Documentation
- [Use Amazon SageMaker Built-in Algorithms](https://docs.aws.amazon.com/sagemaker/latest/dg/algos.html)

### Hands-on examples
- [Get started with Amazon SageMaker](https://aws.amazon.com/sagemaker/getting-started/)


### Workshops
- [Amazon SageMaker 101 Workshop](https://catalog.us-east-1.prod.workshops.aws/workshops/0c6b8a23-b837-4e0f-b2e2-4a3ffd7d645b/en-US)